In [1]:
import pandas as pd
import gc
import main_2_pre_processing
from src.pre_proc import ais_query
import config
import pandas as pd
import gc
import main_2_pre_processing
from src.pre_proc import ais_query
import config
import training_utils
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import os


SEGMENT_MAX_LENGTH = config.SEGMENT_MAX_LENGTH # Maximum length of segments (minimum lenght is set during filtering, and should be equal to this, if we want fixed-length segments)
NUMERIC_COLS = config.NUMERIC_COLS             # Numeric columns to be normalized

TRAIN_START_DATE = config.TRAIN_START_DATE     # Start date for training data you want to PRE-PROCESS
TRAIN_END_DATE = config.TRAIN_END_DATE         # End date for training data you want to PRE-PROCESS


TEST_START_DATE = config.TEST_START_DATE       # Start date for test data you want to PRE-PROCESS
TEST_END_DATE = config.TEST_END_DATE    

NUMERIC_COLS = config.NUMERIC_COLS
FEATURE_COLS = config.FEATURE_COLS

PRE_PROCESSING_DF_TRAIN_PATH = config.PRE_PROCESSING_DF_TRAIN_PATH
PRE_PROCESSING_DF_TEST_PATH = config.PRE_PROCESSING_DF_TEST_PATH

WEIGHTS_PATH = config.WEIGHTS_PATH
PLOT_PATH = config.PLOT_PATH
PREDICTION_DF_PATH = config.PREDICTION_DF_PATH

## Saving pre-processed DF

##### Load data from filtered parquet database

In [3]:
# Filtered AIS data is located at "ais-data/parquet/" and contains your filtered downloaded data (filtered based on bbox, ship types, etc.)
# Still contains NaN values and unprocessed columns
raw_df = ais_query.query_ais_duckdb("ais-data/parquet", verbose=True)
print(f"Raw DataFrame shape: {raw_df.shape}")
print(f"Raw DataFrame columns:\n{raw_df.columns.tolist()}")

[query_ais_duckdb] SQL:
 SELECT * FROM read_parquet('ais-data/parquet/**/*.parquet') WHERE 1=1


MemoryError: Unable to allocate 555. MiB for an array with shape (10, 7276150) and data type object

##### Look at how many records and unique days are in the data

In [2]:
unique_days = pd.Series(raw_df['Timestamp'].dt.normalize().unique())
unique_days = pd.to_datetime(unique_days).sort_values()
print(f"Data loaded: {len(raw_df)} records.")
#print("Unique days:", [d.strftime('%Y-%m-%d') for d in unique_days])

# Print continuous periods (split when a day is missing)
if unique_days.empty:
    print("No days found in data.")
else:
    days = pd.to_datetime(unique_days).sort_values().reset_index(drop=True)
    diffs = days.diff().dt.days
    groups = (diffs.fillna(1) != 1).cumsum()
    periods = [(grp.iloc[0], grp.iloc[-1]) for _, grp in days.groupby(groups)]

    print("Continuous periods (split when a day is missing):")
    for start, end in periods:
        if start == end:
            print(start.strftime("%Y-%m-%d"))
        else:
            length = (end - start).days + 1
            print(f"{start.strftime('%Y-%m-%d')} -> {end.strftime('%Y-%m-%d')} ({length} days)")
    print(f"Total periods: {len(periods)}")


NameError: name 'raw_df' is not defined

In [ ]:
# Free up memory
del raw_df
gc.collect()

##### Pre-processing

In [2]:
# Setup this parameters at your convenience into config.py

SEGMENT_MAX_LENGTH = config.SEGMENT_MAX_LENGTH # Maximum length of segments (minimum lenght is set during filtering, and should be equal to this, if we want fixed-length segments)
NUMERIC_COLS = config.NUMERIC_COLS             # Numeric columns to be normalized

TRAIN_START_DATE = config.TRAIN_START_DATE     # Start date for training data you want to PRE-PROCESS
TRAIN_END_DATE = config.TRAIN_END_DATE         # End date for training data you want to PRE-PROCESS

TEST_START_DATE = config.TEST_START_DATE       # Start date for test data you want to PRE-PROCESS
TEST_END_DATE = config.TEST_END_DATE           # End date for test data you want to PRE-PROCESS

print(f"SEGMENT_MAX_LENGTH: {SEGMENT_MAX_LENGTH}")
print(f"NUMERIC_COLS: {NUMERIC_COLS}")
print(f"TRAIN_START_DATE: {TRAIN_START_DATE}")
print(f"TRAIN_END_DATE: {TRAIN_END_DATE}")
print(f"TEST_START_DATE: {TEST_START_DATE}")
print(f"TEST_END_DATE: {TEST_END_DATE}")

SEGMENT_MAX_LENGTH: 300
NUMERIC_COLS: ['Latitude', 'Longitude', 'SOG', 'COG']
TRAIN_START_DATE: 2025-08-01
TRAIN_END_DATE: 2025-08-31
TEST_START_DATE: 2025-09-01
TEST_END_DATE: 2025-09-03


In [3]:
main_2_pre_processing.main_pre_processing(dataframe_type="train")
# The pre-processed DataFrame is now saved to the path specified in config.PRE_PROCESSING_DF_TRAIN_PATH

# YOU CAN ALSO PRE_PROCESS JUST THE TRAIN DATAFRAME AND THEN SPLIT IT IN TRAIN/TEST IF YOU PREFER

[pre_processing] Querying AIS data for training period: 2025-08-01 to 2025-08-31
[query_ais_duckdb] SQL:
 SELECT * FROM read_parquet('ais-data/parquet/**/*.parquet') WHERE 1=1 AND Date IN ('2025-08-01', '2025-08-02', '2025-08-03', '2025-08-04', '2025-08-05', '2025-08-06', '2025-08-07', '2025-08-08', '2025-08-09', '2025-08-10', '2025-08-11', '2025-08-12', '2025-08-13', '2025-08-14', '2025-08-15', '2025-08-16', '2025-08-17', '2025-08-18', '2025-08-19', '2025-08-20', '2025-08-21', '2025-08-22', '2025-08-23', '2025-08-24', '2025-08-25', '2025-08-26', '2025-08-27', '2025-08-28', '2025-08-29', '2025-08-30', '2025-08-31')
[pre_processing] Initial data size: 3996082 records.
[pre_processing] Dropping unnecessary columns and rows with missing values...
[pre_processing] Data size after dropping: 3925808 records.
[pre_processing] Splitting segments to max length 300...
[pre_processing] Saving pre-processed DataFrame to ais-data/df_preprocessed/pre_processed_df_train.parquet...
[pre_processing] Co

In [3]:
main_2_pre_processing.main_pre_processing(dataframe_type="test")
# The pre-processed DataFrame is now saved to the path specified in config.PRE_PROCESSING_DF_TEST_PATH

[pre_processing] Querying AIS data for testing period: 2025-09-01 to 2025-09-03
[query_ais_duckdb] SQL:
 SELECT * FROM read_parquet('ais-data/parquet/**/*.parquet') WHERE 1=1 AND Date IN ('2025-09-01', '2025-09-02', '2025-09-03')
[pre_processing] Initial data size: 367732 records.
[pre_processing] Dropping unnecessary columns and rows with missing values...
[pre_processing] Data size after dropping: 367230 records.
[pre_processing] Splitting segments to max length 300...
[pre_processing] Saving pre-processed DataFrame to ais-data/df_preprocessed/pre_processed_df_test.parquet...
[pre_processing] Columns of pre-processed DataFrame:
['Timestamp', 'Latitude', 'Longitude', 'SOG', 'COG', 'MMSI', 'Segment_nr', 'ShipTypeID']
[pre_processing] Saving preprocessing metadata to ais-data/df_preprocessed/pre_processing_metadata_test.json...


## Model Loading and training

In [6]:
import pandas as pd
import gc
import main_2_pre_processing
from src.pre_proc import ais_query
import config
import training_utils
from sklearn.model_selection import train_test_split

In [6]:
NUMERIC_COLS = config.NUMERIC_COLS
FEATURE_COLS = config.FEATURE_COLS

PRE_PROCESSING_DF_TRAIN_PATH = config.PRE_PROCESSING_DF_TRAIN_PATH
PRE_PROCESSING_DF_TEST_PATH = config.PRE_PROCESSING_DF_TEST_PATH


# Load pre-processed training DataFrame from file to  ---> sequences
df_seq_train = training_utils.load_df_seq(PRE_PROCESSING_DF_TRAIN_PATH)

df_seq_train, df_seq_val = train_test_split(
        df_seq_train,
        test_size=0.2,
        random_state=5,
    )


# df_seq is a dict with the following structure:   
# {       
#         "Segment_nr": seg_id,
#         "MMSI": mmsi,
#         "ShipTypeID": ship_type_id,
#         "FirstTimestamp": first_timestamp,
#         "Sequence": X,
# })                                                    if you want to modify it, look at training_utils.py load_df_seq function

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch.nn as nn

##### Dataset class

In [14]:
class AISTrajectoryDataset(Dataset):
    """
    Dataset wrapping df_seq:
      - Sequence: list-of-lists (T, F) or np.ndarray
      - ShipTypeID: integer class id
    """

    def __init__(self, df_seq: pd.DataFrame):
        self.df = df_seq.reset_index(drop=True)

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        seq = np.array(row["Sequence"], dtype=np.float32)  # (T, F)
        x = torch.from_numpy(seq)                         # (T, F)

        ship_type_id = int(row["ShipTypeID"])
        ship_type_id = torch.tensor(ship_type_id, dtype=torch.long)

        return x, ship_type_id

In [10]:
train_dataset = AISTrajectoryDataset(df_seq_train)
val_dataset = AISTrajectoryDataset(df_seq_val) # useful for early stopping during training

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    drop_last=False,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=False,
)


# Infer sequence shape and num_shiptypes
sample_x, _ = train_dataset[0]
T, F = sample_x.shape
num_shiptypes = df_seq_train["ShipTypeID"].nunique()
print(f"Sequence shape: T={T}, F={F}, num_shiptypes={num_shiptypes}")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NameError: name 'df_seq_train' is not defined

##### Model class (import your model here)

In [15]:
# Import
from modelAE import LSTMAutoencoderWithShipType

# Instantiate the model
model = LSTMAutoencoderWithShipType(
    input_dim=F,
    hidden_dim=128,
    latent_dim=64,
    num_shiptypes=num_shiptypes,
    shiptype_emb_dim=8,
    num_layers=1,
    dropout=0.3,
).to(device)

NameError: name 'F' is not defined

##### Loss function 


In [2]:
import torch.nn.functional as F

def sequence_loss_fn(
    recon_batch: torch.Tensor,  # (B, T, F)
    x_batch: torch.Tensor,      # (B, T, F)
) -> torch.Tensor:
    """
    Returns loss per sequence: shape (B,).
    Here you can easily change the definition (MSE, MAE, weights, etc.).
    """
    mse = F.mse_loss(recon_batch, x_batch, reduction="none")  # (B, T, F)
    # mean on time and feature, keeping batch separate
    seq_error = mse.mean(dim=(1, 2))  # (B,)
    return seq_error

##### Training

In [8]:
from torch.nn.utils import clip_grad_norm_
max_grad_norm = 1.0 # Maximum gradient norm for gradient clipping (prevent exploding gradients, can be adjusted)

epochs = 10
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


history = {"train_loss": [], "val_loss": []}

for epoch in range(1, epochs + 1):
    # ---- Train ----
    model.train()
    train_loss_sum = 0.0

    for x, ship_type_id in train_loader:
        x = x.to(device)
        ship_type_id = ship_type_id.to(device)

        recon, _ = model(x, ship_type_id)
        seq_errors = sequence_loss_fn(recon, x) 
        loss = seq_errors.mean()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()

        train_loss_sum += loss.item() * x.size(0)

    # Total train loss noemralized by number of samples (a batch could have different size, so we sum and normalize here)
    train_loss = train_loss_sum / len(train_loader.dataset)
    history["train_loss"].append(train_loss)

    # ---- Validation ---- # Useful for early stopping/model selection
    model.eval()
    val_loss_sum = 0.0
    with torch.no_grad():
        for x, ship_type_id in val_loader:
            x = x.to(device)
            ship_type_id = ship_type_id.to(device)

            recon, _ = model(x, ship_type_id)
            seq_errors = sequence_loss_fn(recon, x)
            loss = seq_errors.mean()
            
            val_loss_sum += loss.item() * x.size(0)

    val_loss = val_loss_sum / len(val_loader.dataset)
    history["val_loss"].append(val_loss)

    print(
        f"Epoch {epoch}/{epochs} - "
        f"train MSE: {train_loss:.6f} - val MSE: {val_loss:.6f}"
    )

Epoch 1/10 - train MSE: 0.159706 - val MSE: 0.015657
Epoch 2/10 - train MSE: 0.011318 - val MSE: 0.006591
Epoch 3/10 - train MSE: 0.003598 - val MSE: 0.001126
Epoch 4/10 - train MSE: 0.000710 - val MSE: 0.000477
Epoch 5/10 - train MSE: 0.000387 - val MSE: 0.000328
Epoch 6/10 - train MSE: 0.000319 - val MSE: 0.000346
Epoch 7/10 - train MSE: 0.000285 - val MSE: 0.000254
Epoch 8/10 - train MSE: 0.000248 - val MSE: 0.000301
Epoch 9/10 - train MSE: 0.000246 - val MSE: 0.000205
Epoch 10/10 - train MSE: 0.000187 - val MSE: 0.000166


In [9]:
WEIGHTS_PATH = config.WEIGHTS_PATH
torch.save(model.state_dict(), WEIGHTS_PATH)
print(f"Model weights saved to {WEIGHTS_PATH}")

Model weights saved to models/dark_vessel_model.pth


## Evaluation

In [3]:
import config

WEIGHTS_PATH = config.WEIGHTS_PATH
PLOT_PATH = config.PLOT_PATH
PREDICTION_DF_PATH = config.PREDICTION_DF_PATH

print("WEIGHTS_PATH:", WEIGHTS_PATH)
print("PLOT_PATH:", PLOT_PATH)
print("PREDICTION_DF_PATH:", PREDICTION_DF_PATH)

WEIGHTS_PATH: models/dark_vessel_model.pth
PLOT_PATH: eval/plots
PREDICTION_DF_PATH: eval/val_predictions.parquet


In [4]:
import training_utils

# Load pre-processed TEST DataFrame from file to  ---> sequences
# CAN ALSO BE THE TRAIN/VAL SPLIT IF YOU PREFER, JUST TO SEE HOW IT RECONSTRUCTS KNOWN DATA
df_seq_test = training_utils.load_df_seq(PRE_PROCESSING_DF_TEST_PATH)

In [5]:
# Load test dataset and dataloader
class AISTrajectoryDataset(Dataset):
    """
    Dataset wrapping df_seq:
      - Sequence: list-of-lists (T, F) or np.ndarray
      - ShipTypeID: integer class id
    """

    def __init__(self, df_seq: pd.DataFrame):
        self.df = df_seq.reset_index(drop=True)

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        seq = np.array(row["Sequence"], dtype=np.float32)  # (T, F)
        x = torch.from_numpy(seq)                         # (T, F)

        ship_type_id = int(row["ShipTypeID"])
        ship_type_id = torch.tensor(ship_type_id, dtype=torch.long)

        return x, ship_type_id

test_dataset = AISTrajectoryDataset(df_seq_test)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=False,
)

# Infer sequence shape and num_shiptypes
sample_x, _ = test_dataset[0]
T, F = sample_x.shape
num_shiptypes = df_seq_train["ShipTypeID"].nunique()
print(f"Sequence shape: T={T}, F={F}, num_shiptypes={num_shiptypes}")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NameError: name 'df_seq_train' is not defined

##### Put here your model

In [16]:
# Import
from modelAE import LSTMAutoencoderWithShipType

# Instantiate the model
model = LSTMAutoencoderWithShipType(
    input_dim=F,
    hidden_dim=128,
    latent_dim=64,
    num_shiptypes=num_shiptypes,
    shiptype_emb_dim=8,
    num_layers=1,
    dropout=0.3,
).to(device)

In [17]:
# Load the saved model weights
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=device))
model.to(device)
model.eval() # put model in evaluation mode

LSTMAutoencoderWithShipType(
  (encoder): LSTM(4, 128, batch_first=True)
  (fc_latent): Linear(in_features=128, out_features=64, bias=True)
  (shiptype_emb): Embedding(17, 8)
  (fc_z_st_to_h): Linear(in_features=72, out_features=128, bias=True)
  (decoder): LSTM(4, 128, batch_first=True)
  (fc_out): Linear(in_features=128, out_features=4, bias=True)
)

##### Plots


In [21]:
import evaluation_utils
# Save some plots and get evaluation results about reconstruction errors
results = evaluation_utils.make_plots(model=model, test_loader=test_loader, device=device, seq_loss_fn=sequence_loss_fn)
print("Evaluation results:", results)


=== Reconstruction errors on test ===
Test:
  mean: 0.000228
  std:  0.000564
  q95:  0.000799
  q99:  0.001409
  max:  0.012264

Suggested anomaly threshold (99th percentile of test): 0.001409
Plots saved in directory: "eval/plots"
Evaluation results: {'test_scores': array([1.1595870e-04, 3.2631713e-05, 8.8864195e-05, ..., 2.9249548e-05,
       2.4567564e-05, 1.4920163e-04], shape=(1021,), dtype=float32), 'threshold_99': np.float32(0.0014091902)}


##### Prediction df

In [22]:
# Build predictions DataFrame for further analysis
df_predictions = evaluation_utils.build_predictions_df(model=model, dataset=test_loader.dataset, device=device, seq_loss_fn=sequence_loss_fn)
print("Predictions DataFrame shape:", df_predictions.shape)
print("Predictions DataFrame columns:", df_predictions.columns.tolist())

# Save predictions DataFrame to file
df_predictions.to_parquet(PREDICTION_DF_PATH, index=False)
print(f"Predictions DataFrame saved to {PREDICTION_DF_PATH}")

Predictions DataFrame shape: (1021, 6)
Predictions DataFrame columns: ['Segment_nr', 'MMSI', 'ShipTypeID', 'Sequence_real', 'Sequence_pred', 'recon_error']
Predictions DataFrame saved to eval/val_predictions.parquet


## Inspection

In [17]:
# Free up memory
del df_seq_train
del df_seq_val
del df_seq_test
del df_predictions
gc.collect()

20

##### Denormalization

In [7]:
import inspection_utils
import pandas as pd
from inspection_utils import denormalize_predictions

import pandas as pd
import gc
import main_2_pre_processing
from src.pre_proc import ais_query
import config
import pandas as pd
import gc
import main_2_pre_processing
from src.pre_proc import ais_query
import config
import training_utils
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import os


SEGMENT_MAX_LENGTH = config.SEGMENT_MAX_LENGTH # Maximum length of segments (minimum lenght is set during filtering, and should be equal to this, if we want fixed-length segments)
NUMERIC_COLS = config.NUMERIC_COLS             # Numeric columns to be normalized

TRAIN_START_DATE = config.TRAIN_START_DATE     # Start date for training data you want to PRE-PROCESS
TRAIN_END_DATE = config.TRAIN_END_DATE         # End date for training data you want to PRE-PROCESS


TEST_START_DATE = config.TEST_START_DATE       # Start date for test data you want to PRE-PROCESS
TEST_END_DATE = config.TEST_END_DATE    

NUMERIC_COLS = config.NUMERIC_COLS
FEATURE_COLS = config.FEATURE_COLS

PRE_PROCESSING_DF_TRAIN_PATH = config.PRE_PROCESSING_DF_TRAIN_PATH
PRE_PROCESSING_DF_TEST_PATH = config.PRE_PROCESSING_DF_TEST_PATH

WEIGHTS_PATH = config.WEIGHTS_PATH
PLOT_PATH = config.PLOT_PATH
PREDICTION_DF_PATH = config.PREDICTION_DF_PATH

df_predictions = pd.read_parquet(PREDICTION_DF_PATH)
# print("Loaded Predictions DataFrame shape:", df_predictions.shape)
# print(df_predictions.head().to_string)


df_denormalized = denormalize_predictions(
    df=df_predictions,
    metadata_path=config.PRE_PROCESSING_METADATA_TEST_PATH,
)

# print("Denormalized Predictions DataFrame head: ")
# print(df_denormalized.head())





>>> raw normalized real_lon (prima della denorm)
[0.006969360169023275, 0.008903004229068756, 0.010337387211620808, 0.012496885843575, 0.014291846193373203, 0.016035296022892, 0.017628174275159836, 0.019609365612268448, 0.021055635064840317, 0.023064564913511276, 0.025025945156812668, 0.026832792907953262, 0.028453407809138298, 0.030398936942219734, 0.03180558234453201, 0.03374319151043892, 0.03573627024888992, 0.03573627024888992, 0.037578776478767395, 0.039254866540431976, 0.04125586897134781, 0.043034981936216354, 0.04463182017207146, 0.04659320041537285, 0.048404011875391006, 0.05004047602415085, 0.05200185626745224, 0.05379285290837288, 0.05543724447488785, 0.05743032321333885, 0.059225283563137054, 0.06088552251458168, 0.0628984123468399, 0.06470922380685806, 0.06637738645076752, 0.07024863362312317, 0.07188510149717331, 0.07575634866952896, 0.07740073651075363, 0.07917588949203491, 0.08116500079631805, 0.0828014686703682, 0.08462416380643845, 0.08665290474891663, 0.0883052200078

##### Plot on map

In [8]:
import inspection_utils
import config
MAPS_PATH = config.MAPS_PATH

# un singolo segmento
#inspection_utils.save_interactive_html(df_denormalized, out_html=f"{MAPS_PATH}/ais_maps.html", segment=7235, zoom_start=7)

# random 8 tracce
inspection_utils.save_interactive_html(df_denormalized, out_html=f"{MAPS_PATH}/ais_maps.html", n_random=8, zoom_start=7)

# un singolo MMSI
#inspection_utils.save_interactive_html(df_denormalized, out_html=f"{MAPS_PATH}/ais_mmsi_219005866.html", mmsi=219005866, zoom_start=8)

# lista di MMSI
#inspection_utils.save_interactive_html(df_denormalized, out_html=f"{MAPS_PATH}/ais_some_mmsi.html", mmsi=[219005866, 241455000], zoom_start=8)

# head 10
df_denormalized = df_denormalized.sort_values(by=["recon_error"]) 
inspection_utils.save_interactive_html(df_denormalized, out_html=f"{MAPS_PATH}/ais_best_first_10.html", head_n=10)
# head 10
df_denormalized = df_denormalized.sort_values(by=["recon_error"], ascending=False)
inspection_utils.save_interactive_html(df_denormalized, out_html=f"{MAPS_PATH}/ais_worst_first_10.html", head_n=10)

'eval/maps/ais_worst_first_10.html'